# This task is not quite ready as we don't have an open source route for simulating geometry that requires imprinting and merging. However this simulation can be carried out using Trelis.

# Heating Mesh Tally on CAD geometry made from Components

This constructs a reactor geometry from 3 Component objects each made from points.

The Component made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This section makes the 3d geometry for the entire reactor from a input parameters.

In [ ]:
import paramak

my_reactor = paramak.BallReactor(
    inner_bore_radial_thickness=50,
    inboard_tf_leg_radial_thickness=55,
    center_column_shield_radial_thickness=50,
    divertor_radial_thickness=50,
    inner_plasma_gap_radial_thickness=50,
    plasma_radial_thickness=100,
    outer_plasma_gap_radial_thickness=50,
    firstwall_radial_thickness=1,
    blanket_radial_thickness=100,
    blanket_rear_wall_radial_thickness=10,
    elongation=2,
    triangularity=0.55,
    number_of_tf_coils=16,
    rotation_angle=180,
)

# TF and PF coils can be added with additional arguments.
# see the documentation for more details 
# https://paramak.readthedocs.io/en/main/paramak.parametric_reactors.html

my_reactor.show()

Exports the 3D geometry to a DAGMC neutronics geometry. The plasma is not included as not many neutron interactions occur in the low density plasma

In [ ]:
my_reactor.export_dagmc_h5m('dagmc_3.h5m', exclude='plasma')

The next section defines the materials. This can be done using openmc.Materials or in this case strings that look up materials from the neutronics material maker.

In [ ]:
from neutronics_material_maker import Material

mat1 = Material.from_library(name='Li4SiO4')

mat2 = Material.from_library(name='copper')

mat3 = Material.from_library(name='WC')

mat4 = Material.from_library(name='eurofer')

This next section combines the geometry with the materials and specifies a few mesh tallies

In [ ]:
# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. 

import openmc_dagmc_wrapper as odw
import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package
material_tag_to_material_dict = {
    'mat_blanket': mat1,
    'mat_blanket_rear_wall':mat4,
    'mat_center_column_shield':mat3,
    'mat_divertor_lower':mat4,
    'mat_divertor_upper':mat4,
    'mat_firstwall':mat4,
    'mat_inboard_tf_coils':mat2,
    'mat_plasma':mat2 # todo remove
}

geometry = odw.Geometry(h5m_filename='dagmc_3.h5m')

materials = odw.Materials(
    h5m_filename='dagmc_3.h5m',
    correspondence_dict=material_tag_to_material_dict
)

This next step makes a simple point source.

In [ ]:

import openmc

# initialises a new source object
source = openmc.Source()

# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((100, 0, 0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

Makes a 3D mesh tally

starts the simulation

extracts the 3d mesh tally result and plots it as a vtk

In [ ]:
# from openmc_mesh_tally_to_vtk import write_mesh_tally_to_vtk
# import openmc

# # assumes you have a statepoint file from the OpenMC simulation
# statepoint = openmc.StatePoint('statepoint.3.h5')

# # assumes the statepoint file has a RegularMesh tally with a certain name
# my_tally = statepoint.get_tally(name='tally_on_regular_mesh')

# # converts the tally result into a VTK file
# write_mesh_tally_to_vtk(
#     tally=my_tally,
#     filename = "vtk_file_from_openmc_mesh.vtk",
# )

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from IPython.display import FileLink
display(FileLink('heating_on_3D_mesh.vtk'))